##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 合成グラフを使ってセンチメント分類を実施するためのグラフ正則化

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
  <td><a href="https://tfhub.dev/"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TFハブモデルを参照してください</a></td>
</table>

## 概要

このノートブックでは、映画レビューのテキストを使用して、それが*肯定的*であるか*否定的*であるかに分類します。これは*二項*分類の例で、機械学習問題では重要な分類法として広く適用されています。

このノートブックでは、特定の入力からグラフを構築することでグラフ正則化の使用方法を示しています。入力に明示的なグラフが含まれない場合に、Neural Structured Learning（NSL）フレームワークを使ってグラフ正則化モデルを構築するための一般的なレシピは以下のとおりです。

1. 入力のテキストサンプルに埋め込みを作成します。これは、[word2vec](https://arxiv.org/pdf/1310.4546.pdf)、[Swivel](https://arxiv.org/abs/1602.02215)、[BERT](https://arxiv.org/abs/1810.04805) などの事前トレーニング済みのモデルを使って行えます。
2. 'L2' distance、'cosine' distance などの類似度メトリクスを使って、これらの埋め込みに基づくグラフを構築します。グラフ内のノードはサンプルに対応し、グラフ内のエッジはサンプルペア間の類似度に対応します。
3. 上記の合成グラフとサンプル特徴量からトレーニングデータを生成します。生成されたトレーニングデータには、元のノード特徴量のほかに、近傍する特徴量が含まれます。
4. Keras Sequential API、Functional API、または Subclass API を使用して、基本モデルとしてニューラルネットワークを作成します。
5. NSL フレームワークが提供する GraphRegularization ラッパークラスで基本モデルをラップし、新しいグラフ Keras モデルを作成します。この新しいモデルは、トレーニング目的の正則化項にグラフ正則化損失を含みます。
6. グラフ Keras モデルをトレーニングして評価します。

**注意**: このチュートリアルにかかる時間はおよそ 1 時間を想定しています。

## 要件

1. Neural Structured Learning パッケージをインストールします。
2. tensorflow-hub をインストールします。

In [ ]:
!pip install --quiet neural-structured-learning
!pip install --quiet tensorflow-hub

## 依存関係とインポート

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import neural_structured_learning as nsl

import tensorflow as tf
import tensorflow_hub as hub

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## IMDB データセット

[IMDB データセット](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)には、[Internet Movie Database](https://www.imdb.com/) から抽出した 50,000 件の映画レビューのテキストが含まれています。これらはトレーニング用とテスト用に 25,000 件ずつに分割されています。トレーニング用とテスト用のセットは*均衡*しています。つまり、これらに含まれる肯定的なレビューと否定的なレビューの件数が同数であるということです。

このチュートリアルでは、事前処理済みの IMDB データセットを使用します。

### 事前処理済みの IMDB データセットをダウンロードする

IMDB データセットは TensorFlow にパッケージ化されています。事前に処理済みであるため、レビュー（単語のシーケンス）は整数のシーケンスに変換されています。各整数は、ディクショナリ内の特定の単語を表現します。

以下のコードを使って IMDB データセットをダウンロードします（または、ダウンロード済みの場合はキャッシュされたコピーを使用します）。

In [ ]:
imdb = tf.keras.datasets.imdb
(pp_train_data, pp_train_labels), (pp_test_data, pp_test_labels) = (
    imdb.load_data(num_words=10000))

引数 `num_words=10000` によって、トレーニングデータ内で最も頻繁に出現する単語の上位 10,000 語が保持されます。語彙を管理しやすいサイズに維持するために、まれに出現する単語は破棄されます。

### データを調べる

データの形式を確認してみましょう。データセットは事前処理が行われているため、各サンプルは、映画レビューの単語を表現する整数の配列です。各ラベルは 0 または 1 の整数値で、0 は否定的なレビュー、1 は肯定的なレビューを示します。

In [ ]:
print('Training entries: {}, labels: {}'.format(
    len(pp_train_data), len(pp_train_labels)))
training_samples_count = len(pp_train_data)

レビューのテキストは整数に変換済みであり、これらの各整数はディクショナリ内の特定の単語を表現します。以下は、最初のレビューです。

In [ ]:
print(pp_train_data[0])

映画レビューの長さは異なります。以下のコードでは、1 番目と 2 番目の映画レビューの語数を示します。ニューラルネットワークへの入力は同じ長さである必要があるため、これについて後で解決する必要があります。

In [ ]:
len(pp_train_data[0]), len(pp_train_data[1])

### 整数を単語に変換し直す

整数を対応するテキストに変換する方法を知っておくと便利かもしれません。ここでは、整数と文字列のマッピングを含むディクショナリオブジェクトをクエリするヘルパー関数を作成することにします。

In [ ]:
def build_reverse_word_index():
  # A dictionary mapping words to an integer index
  word_index = imdb.get_word_index()

  # The first indices are reserved
  word_index = {k: (v + 3) for k, v in word_index.items()}
  word_index['<PAD>'] = 0
  word_index['<START>'] = 1
  word_index['<UNK>'] = 2  # unknown
  word_index['<UNUSED>'] = 3
  return dict((value, key) for (key, value) in word_index.items())

reverse_word_index = build_reverse_word_index()

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

これで `decode_review` 関数を使用して、最初のレビューのテキストを表示できるようになりました。

In [ ]:
decode_review(pp_train_data[0])

## グラフの構築

グラフの構築では、テキストサンプル用の埋め込みを作成してから、類似度関数を使って埋め込みの比較が行われます。

先に進む前にまず、このチュートリアルで作成されるアーティファクトを保存するためのディクショナリを作成します。

In [ ]:
!mkdir -p /tmp/imdb

### サンプルの埋め込みを作成する

事前トレーニング済みの Swivel 埋め込みを使用して、入力の各サンプルに使用する埋め込みを `tf.train.Example` 形式で作成します。作成した埋め込みを各サンプルの ID とともに `TFRecord` 形式で保存します。これは重要な手順であり、後でサンプルの埋め込みと対応するノードをグラフで一致させることができます。

In [ ]:
pretrained_embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'

hub_layer = hub.KerasLayer(
    pretrained_embedding, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
def _int64_feature(value):
  """Returns int64 tf.train.Feature."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value.tolist()))


def _bytes_feature(value):
  """Returns bytes tf.train.Feature."""
  return tf.train.Feature(
      bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))


def _float_feature(value):
  """Returns float tf.train.Feature."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value.tolist()))


def create_embedding_example(word_vector, record_id):
  """Create tf.Example containing the sample's embedding and its ID."""

  text = decode_review(word_vector)

  # Shape = [batch_size,].
  sentence_embedding = hub_layer(tf.reshape(text, shape=[-1,]))

  # Flatten the sentence embedding back to 1-D.
  sentence_embedding = tf.reshape(sentence_embedding, shape=[-1])

  features = {
      'id': _bytes_feature(str(record_id)),
      'embedding': _float_feature(sentence_embedding.numpy())
  }
  return tf.train.Example(features=tf.train.Features(feature=features))


def create_embeddings(word_vectors, output_path, starting_record_id):
  record_id = int(starting_record_id)
  with tf.io.TFRecordWriter(output_path) as writer:
    for word_vector in word_vectors:
      example = create_embedding_example(word_vector, record_id)
      record_id = record_id + 1
      writer.write(example.SerializeToString())
  return record_id


# Persist TF.Example features containing embeddings for training data in
# TFRecord format.
create_embeddings(pp_train_data, '/tmp/imdb/embeddings.tfr', 0)

### グラフを構築する

サンプルの埋め込みを作成したので、それを使用して類似度グラフを構築します。つまり、このグラフのノードはサンプルに対応し、エッジはノードペア間の類似度に対応します。

Neural Structured Learning にはグラフ構築用のライブラリが備わっており、サンプルの埋め込みに基づいてグラフを作成することができます。類似度の測定として[**コサイン類似度**](https://en.wikipedia.org/wiki/Cosine_similarity)を使用して埋め込みを比較し、それらの間にエッジを作成します。また、類似度のしきい値を指定できるため、それを使用して、類似しないエッジを最終グラフから破棄することができます。次の例では、0.99 を類似度のしきい値として、12345 をランダムシードとして使用し、429,415 個の双方向エッジを持つグラフが構築されています。ここでは、グラフビルダーの[局所性鋭敏型ハッシュ](https://en.wikipedia.org/wiki/Locality-sensitive_hashing)（LSH）サポートを使用して、グラフの構築を高速化しています。グラフビルダーの LSH サポートについては、[`build_graph_from_config`](https://www.tensorflow.org/neural_structured_learning/api_docs/python/nsl/tools/build_graph_from_config) API ドキュメントをご覧ください。

In [ ]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=0.99, lsh_splits=32, lsh_rounds=15, random_seed=12345)
nsl.tools.build_graph_from_config(['/tmp/imdb/embeddings.tfr'],
                                  '/tmp/imdb/graph_99.tsv',
                                  graph_builder_config)

それぞれの双方向エッジは、出力 TSV ファイルの 2 つの有向エッジで表現されているため、ファイルには、合計 429,415 * 2 = 858,830 行が含まれます。

In [ ]:
!wc -l /tmp/imdb/graph_99.tsv

**注意:** グラフの品質と、その延長として埋め込みの品質はグラフの正則化において非常に重要です。このノードブックでは Swivel 埋め込みを使用しましたが、たとえば BERT 埋め込みを使用した場合には、レビューのセマンティクスをより正確に捉えられる可能性があります。それぞれのニーズに合ったさまざまな埋め込みを使用することをお勧めします。

## サンプル特徴量

この問題のサンプル特徴量を `tf.train.Example` 形式で作成し、`TFRecord` 形式で永続化します。各サンプルには、次の 3 つの特徴量が含まれます。

1. **id**: サンプルのノード ID。
2. **words**: 単語 ID を含む int64 リスト。
3. **label**: レビューのターゲットクラスを識別するシングルトン int64。

In [ ]:
def create_example(word_vector, label, record_id):
  """Create tf.Example containing the sample's word vector, label, and ID."""
  features = {
      'id': _bytes_feature(str(record_id)),
      'words': _int64_feature(np.asarray(word_vector)),
      'label': _int64_feature(np.asarray([label])),
  }
  return tf.train.Example(features=tf.train.Features(feature=features))

def create_records(word_vectors, labels, record_path, starting_record_id):
  record_id = int(starting_record_id)
  with tf.io.TFRecordWriter(record_path) as writer:
    for word_vector, label in zip(word_vectors, labels):
      example = create_example(word_vector, label, record_id)
      record_id = record_id + 1
      writer.write(example.SerializeToString())
  return record_id

# Persist TF.Example features (word vectors and labels) for training and test
# data in TFRecord format.
next_record_id = create_records(pp_train_data, pp_train_labels,
                                '/tmp/imdb/train_data.tfr', 0)
create_records(pp_test_data, pp_test_labels, '/tmp/imdb/test_data.tfr',
               next_record_id)

## グラフ近傍を使ってトレーニングデータを拡張する

サンプルの特徴量と合成したグラフがあるため、Neural Structured Learning 用の拡張トレーニングデータを生成することができます。NSL フレームワークにはグラフとサンプル特徴量を合成し、最終的なトレーニングデータを作成してグラフの正則化を得るためのライブラリがあります。作成されたトレーニングデータには元のサンプル特徴量とそれに対応する近傍値が含まれます。

このチュートリアルでは、無向エッジを考慮し、サンプルごとに最大 3 つの近傍値を使用して、グラフ近傍でトレーニングデータを拡張します。

In [ ]:
nsl.tools.pack_nbrs(
    '/tmp/imdb/train_data.tfr',
    '',
    '/tmp/imdb/graph_99.tsv',
    '/tmp/imdb/nsl_train_data.tfr',
    add_undirected_edges=True,
    max_nbrs=3)

## 基本モデル

グラフの正則化を行わずに基本モデルを構築する準備が整いました。このモデルを構築するには、グラフを構築するために使用された埋め込みを使用するか、分類タスクと同時に新しい埋め込みを学習することができます。このノートブックの目的により、ここでは後者を行うことにします。

### グローバル変数

In [ ]:
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

### ハイパーパラメータ

`HParams` のインスタンスを使用して、トレーニングと評価に使用する様々なハイパーパラメータと定数をインクルードします。それぞれについての簡単な説明を以下に示します。

- **num_classes**: *positive* と *negative* の 2 つのクラスがあります。

- **max_seq_length**: これはこの例のそれぞれの映画レビューから考慮される単語の最大数です。

- **vocab_size**: これは、この例で考慮される語彙のサイズです。

- **distance_type**：これはサンプルをその近傍と正則化する際に使用する距離メトリックです。

- **graph_regularization_multiplier**：これは全体の損失関数においてグラフ正則化項の相対的な重みを制御します。

- **num_neighbors**: グラフ正則化に使用する近傍の数です。この値は、`nsl.tools.pack_nbrs` を呼び出す際に上記で使用した `max_nbrs` 以下である必要があります。

- **num_fc_units**: ニューラルネットワークの全結合レイヤーのユニット数。

- **train_epochs**：トレーニングのエポック数。

- **<code>batch_size</code>**: トレーニングや評価に使用するバッチサイズ。

- **eval_steps**：評価が完了したと判断するまでに処理を行うバッチ数。`None` 設定にすると、テストセット内の全てのインスタンスを評価します。

In [ ]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    self.num_classes = 2
    self.max_seq_length = 256
    self.vocab_size = 10000
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 2
    ### model architecture
    self.num_embedding_dims = 16
    self.num_lstm_dims = 64
    self.num_fc_units = 64
    ### training parameters
    self.train_epochs = 10
    self.batch_size = 128
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

### データを準備する

整数の配列で表現されたレビューをニューラルネットワークにフィードする前にテンソルに変換する必要があります。この変換は、以下の 2 つの方法で行われます。

- 配列を、ワンホットエンコーディングと同様に、単語の出現を示す `0` と `1` のベクトルに変換します。たとえば、シーケンス `[3, 5]` は、1 を示す `3` と `5` を除き、すべてゼロの `10000` 次元のベクトルになります。次に、これをネットワークの最初のレイヤーである、浮動小数点のベクトルデータを処理できる `Dense` レイヤーにします。ただし、このアプローチはメモリを集中的に使用するため、`num_words * num_reviews`     サイズの行列が必要です。

- または、配列の長さが同じになるように配列にパディングを行い、形状 `max_length * num_reviews` の整数テンソルを作成することができます。この形状を処理できる埋め込みレイヤーをネットワークの最初のレイヤーとして使用します。

このチュートリアルでは、後者のアプローチを使用します。

映画レビューの長さは同じである必要があるため、以下に定義される `pad_sequence` 関数を使用して、長さを標準化します。

In [ ]:
def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """

  def pad_sequence(sequence, max_seq_length):
    """Pads the input sequence (a `tf.SparseTensor`) to `max_seq_length`."""
    pad_size = tf.maximum([0], max_seq_length - tf.shape(sequence)[0])
    padded = tf.concat(
        [sequence.values,
         tf.fill((pad_size), tf.cast(0, sequence.dtype))],
        axis=0)
    # The input sequence may be larger than max_seq_length. Truncate down if
    # necessary.
    return tf.slice(padded, [0], [max_seq_length])

  def parse_example(example_proto):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth labels.
    """
    # The 'words' feature is a variable length word ID vector.
    feature_spec = {
        'words': tf.io.VarLenFeature(tf.int64),
        'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1),
    }
    # We also extract corresponding neighbor features in a similar manner to
    # the features above during training.
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i,
                                         NBR_WEIGHT_SUFFIX)
        feature_spec[nbr_feature_key] = tf.io.VarLenFeature(tf.int64)

        # We assign a default value of 0.0 for the neighbor weight so that
        # graph regularization is done on samples based on their exact number
        # of neighbors. In other words, non-existent neighbors are discounted.
        feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
            [1], tf.float32, default_value=tf.constant([0.0]))

    features = tf.io.parse_single_example(example_proto, feature_spec)

    # Since the 'words' feature is a variable length word vector, we pad it to a
    # constant maximum length based on HPARAMS.max_seq_length
    features['words'] = pad_sequence(features['words'], HPARAMS.max_seq_length)
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        features[nbr_feature_key] = pad_sequence(features[nbr_feature_key],
                                                 HPARAMS.max_seq_length)

    labels = features.pop('label')
    return features, labels

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset('/tmp/imdb/nsl_train_data.tfr', True)
test_dataset = make_dataset('/tmp/imdb/test_data.tfr')

### モデルを構築する

ニューラルネットワークは、レイヤーをスタックして作成されており、これには、2 つの主なアーキテクチャ上の決定が必要です。

- モデルにはいくつのレイヤーを使用するか。
- 各レイヤーにはいくつの*非表示ユニット*を使用するか。

この例では、入力データは単語のインデックスの配列で構成されています。予測するラベルは 0 または 1 です。

このチュートリアルでは、基本モデルとして双方向 LSTM を使用します。

In [ ]:
# This function exists as an alternative to the bi-LSTM model used in this
# notebook.
def make_feed_forward_model():
  """Builds a simple 2 layer feed forward neural network."""
  inputs = tf.keras.Input(
      shape=(HPARAMS.max_seq_length,), dtype='int64', name='words')
  embedding_layer = tf.keras.layers.Embedding(HPARAMS.vocab_size, 16)(inputs)
  pooling_layer = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
  dense_layer = tf.keras.layers.Dense(16, activation='relu')(pooling_layer)
  outputs = tf.keras.layers.Dense(1)(dense_layer)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def make_bilstm_model():
  """Builds a bi-directional LSTM model."""
  inputs = tf.keras.Input(
      shape=(HPARAMS.max_seq_length,), dtype='int64', name='words')
  embedding_layer = tf.keras.layers.Embedding(HPARAMS.vocab_size,
                                              HPARAMS.num_embedding_dims)(
                                                  inputs)
  lstm_layer = tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(HPARAMS.num_lstm_dims))(
          embedding_layer)
  dense_layer = tf.keras.layers.Dense(
      HPARAMS.num_fc_units, activation='relu')(
          lstm_layer)
  outputs = tf.keras.layers.Dense(1)(dense_layer)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


# Feel free to use an architecture of your choice.
model = make_bilstm_model()
model.summary()

レイヤーは分類器を構成するため効果的に一列に積み重ねられます。

1. 最初のレイヤーは、整数でエンコーディングされた語彙を取る `Input` レイヤーです。
2. 次のレイヤーは、整数でエンコーディングされた語彙を受け取って、埋め込みベクトルで各単語インデックスをルックアップする `Embedding` レイヤーです。これらのベクトルはモデルのトレーニングの過程で学習されます。ベクトルは出力配列に次元を追加します。生成される次元は、`(batch, sequence, embedding)` です。
3. 次に、双方向 LSTM レイヤーがサンプルごとに固定長の出力ベクトルを返します。
4. この固定長の出力ベクトルは、64 個の非表示ユニットを持つ全結合（`Dense`）レイヤーに受け渡されます。
5. 最後のレイヤーは、単一の出力ノードに密に接続されています。`sigmoid` 活性化関数を使用し、この値は、確率または信頼水準を表す 0 と 1 の間の浮動小数となります。

### 非表示ユニット

上記のモデルには、`Embedding` を除き、入力と出力の間に 2 つの中間または「非表示」レイヤーがあります。出力数（ユニット、ノード、またはニューロン）はレイヤーの表現空間の次元で、言い換えると、内部表現を学習する際にネットワークに許可された自由度です。

モデルにより大きい非表示ユニット数（より高次元の表現空間）がある場合や、レイヤー数が増えるほど、ネットワークはよく複雑な表現を学習できますが、ネットワークの計算コストが高まり、不要なパターンが学習される可能性があります。これらのパターンはトレーニングデータのパフォーマンスを改善しても、テストデータのパフォーマンスは改善しません。この現象は「*過適合*」と呼ばれています。

### 損失関数とオプティマイザ

モデルをトレーニングするには、損失関数とオプティマイザが必要です。これは二項分類問題であり、モデルは確率（シグモイド活性を持つ単一ユニットレイヤー）を出力するため、`binary_crossentropy` 損失関数を使用します。

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### 検証セットを作成する

トレーニングの際に、モデルが遭遇したことのないデータで、モデルの精度を確認します。この場合、元のトレーニングデータの一部を分割し、*検証セット*を作成します。（ここでテストセットを使用しないのは、トレーニングデータのみを使用してモデルの開発とチューニングを行い、その後でテストデータを一度だけ使用して精度を評価することが目標であるためです）。

このチュートリアルでは、最初のトレーニングサンプルのおよそ 10%（25000 の 10%）をトレーニングのラベル付きデータとして取り、残りを検証データとしています。最初のトレーニングデータとテストデータの割合は 50:50（それぞれ 25000 個のサンプル）であったため、実際のトレーニング/検証/テストの分割率は、5:45:50 です。

'train_dataset' にはすでにバッチ化とシャッフルが行われいます。 

In [ ]:
validation_fraction = 0.9
validation_size = int(validation_fraction *
                      int(training_samples_count / HPARAMS.batch_size))
print(validation_size)
validation_dataset = train_dataset.take(validation_size)
train_dataset = train_dataset.skip(validation_size)

### モデルをトレーニングする

モデルをミニバッチでトレーニングします。トレーニング中に、検証セットでのモデルの損失と精度を監視します。

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### モデルを評価する

モデルがどのように実行するかを確認しましょう。損失（誤差を表す数値で、低いほど良です）と精度の 2 つの値が返されます。

In [ ]:
results = model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(results)

### 精度と損失の経時的なグラフを作成する

`model.fit()` は、トレーニング中に発生したすべての情報を収めたディクショナリを含む `History` オブジェクトを返します。

In [ ]:
history_dict = history.history
history_dict.keys()

トレーニングと検証中に監視されている各メトリックに対して 1 つずつ、計 4 つのエントリがあります。このエントリを使用して、トレーニングと検証の損失とトレーニングと検証の精度を比較したグラフを作成することができます。

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

トレーニングの損失がエポックごとに*下降*し、トレーニングの精度がエポックごとに*上昇*していることに注目してください。これは、勾配下降最適化を使用しているときに見られる現象で、イテレーションごとに希望する量を最小化します。

## グラフの正則化

上記で構築した基本モデルを使用して、グラフの正則化を試す準備が整いました。Neural Structured Learning フレームワークが提供する `GraphRegularization` ラッパークラスを使用して基本（bi-LSTM）モデルをラップし、グラフの正則化を含めます。グラフ正則化のトレーニングと評価の残りのステップは、基本モデルのトレーニングと評価と同じです。

### グラフ正則化モデルを作成する

グラフ正則化の増分効果を評価するために、基本モデルの新しいインスタンスを作成します。これは、`model` がすでに数回のイテレーションでトレーニングされており、このトレーニング済みのモデルを再利用してグラフ正則化モデルを作成しても、`model`の公平な比較にならないためです。

In [ ]:
# Build a new base LSTM model.
base_reg_model = make_bilstm_model()

In [ ]:
# Wrap the base model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### モデルをトレーニングする

In [ ]:
graph_reg_history = graph_reg_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### モデルを評価する

In [ ]:
graph_reg_results = graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(graph_reg_results)

### 精度と損失の経時的なグラフを作成する

In [ ]:
graph_reg_history_dict = graph_reg_history.history
graph_reg_history_dict.keys()

ディクショナリには、トレーニング損失、トレーニング精度、トレーニンググラフ損失、検証損失、および検証精度の 5 つのエントリがあります。これらをまとめてプロットし、比較に使用することができます。グラフ損失はトレーニング中にのみ計算されることに注意してください。

In [ ]:
acc = graph_reg_history_dict['accuracy']
val_acc = graph_reg_history_dict['val_accuracy']
loss = graph_reg_history_dict['loss']
graph_loss = graph_reg_history_dict['scaled_graph_loss']
val_loss = graph_reg_history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.clf()   # clear figure

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-gD" is for solid green line with diamond markers.
plt.plot(epochs, graph_loss, '-gD', label='Training graph loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

## 半教師あり学習の性能

半教師あり学習、さらに具体的に言えば、このチュートリアルの文脈でのグラフ正則化は、トレーニングデータの量が少ない場合に非常に強力です。トレーニングデータの不足分は、トレーニングサンプル間の類似度を利用して補完されます。これは、従来の教師あり学習では実現できません。

***supervision ratio***（教師率）を、トレーニング、検証、およびテストサンプルを含むサンプル総数に対するトレーニングサンプルの比率として定義します。このノートブックでは、基本モデルとグラフ正則化モデルの両方のトレーニングに 0.05 の教師率（ラベル付きデータの 5%）を使用しました。教師率がモデルの精度に与える影響を以下のセルで説明します。

In [ ]:
# Accuracy values for both the Bi-LSTM model and the feed forward NN model have
# been precomputed for the following supervision ratios.

supervision_ratios = [0.3, 0.15, 0.05, 0.03, 0.02, 0.01, 0.005]

model_tags = ['Bi-LSTM model', 'Feed Forward NN model']
base_model_accs = [[84, 84, 83, 80, 65, 52, 50], [87, 86, 76, 74, 67, 52, 51]]
graph_reg_model_accs = [[84, 84, 83, 83, 65, 63, 50],
                        [87, 86, 80, 75, 67, 52, 50]]

plt.clf()  # clear figure

fig, axes = plt.subplots(1, 2)
fig.set_size_inches((12, 5))

for ax, model_tag, base_model_acc, graph_reg_model_acc in zip(
    axes, model_tags, base_model_accs, graph_reg_model_accs):

  # "-r^" is for solid red line with triangle markers.
  ax.plot(base_model_acc, '-r^', label='Base model')
  # "-gD" is for solid green line with diamond markers.
  ax.plot(graph_reg_model_acc, '-gD', label='Graph-regularized model')
  ax.set_title(model_tag)
  ax.set_xlabel('Supervision ratio')
  ax.set_ylabel('Accuracy(%)')
  ax.set_ylim((25, 100))
  ax.set_xticks(range(len(supervision_ratios)))
  ax.set_xticklabels(supervision_ratios)
  ax.legend(loc='best')

plt.show()

教師率が低下すると、モデルの精度も低下することが観測できます。これは、使用されているモデルのアーキテクチャに関係なく、基本モデルとグラフ正則化モデルの両方に当てはまることです。ただし、両方のアーキテクチャにおいて、グラフ正則化モデルのパフォーマンスが基本モデルよりも高いことに注目してください。特に Bi-LSTM モデルの場合、教師率が 0.01 の時のグラフ正則化モデルの精度は、基本モデルの精度よりも **約 20%** 高くなっています。これは主に、グラフ正則化モデルの半教師あり学習において、トレーニングサンプルのほかにトレーニングサンプル間の構造的類似度が使用されたためです。

## 結論

入力に明示的なグラフが含まれていない場合でも、Newral Structured Leaning（NSL）フレームワークを使用したグラフ正則化の使用を説明しました。IMDB 映画レビューのセンチメント分類タスクを考察する上で、レビューの埋め込みに基づく類似度グラフを合成しました。ハイパーパラメータや教師率を変化させ、異なるモデルアーキテクチャを使用することで、さらに実験することをお勧めします。